Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 0.02
graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Loss function using L2 Regularization
  regularizer = tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(loss + beta * regularizer)

  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))

      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 79.984665
Training accuracy: 7.6%
Validation accuracy: 9.6%
Loss at step 100: 8.662938
Training accuracy: 75.0%
Validation accuracy: 75.0%
Loss at step 200: 1.729408
Training accuracy: 81.7%
Validation accuracy: 80.8%
Loss at step 300: 0.888989
Training accuracy: 82.9%
Validation accuracy: 81.7%
Loss at step 400: 0.784203
Training accuracy: 83.0%
Validation accuracy: 81.8%
Loss at step 500: 0.770739
Training accuracy: 83.0%
Validation accuracy: 82.0%
Loss at step 600: 0.768903
Training accuracy: 82.9%
Validation accuracy: 82.0%
Loss at step 700: 0.768604
Training accuracy: 82.9%
Validation accuracy: 82.0%
Loss at step 800: 0.768531
Training accuracy: 83.0%
Validation accuracy: 82.0%
Test accuracy: 88.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [46]:
batch_size = 5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Loss function using L2 Regularization
  regularizer = tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(loss + beta * regularizer)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [48]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run( 
      [optimizer, loss, train_prediction], feed_dict=feed_dict )
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy( predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 50.923584
Minibatch accuracy: 20.0%
Validation accuracy: 15.5%
Minibatch loss at step 500: 2.295671
Minibatch accuracy: 80.0%
Validation accuracy: 69.8%
Minibatch loss at step 1000: 5.294910
Minibatch accuracy: 40.0%
Validation accuracy: 68.5%
Minibatch loss at step 1500: 5.329885
Minibatch accuracy: 80.0%
Validation accuracy: 61.4%
Minibatch loss at step 2000: 8.525631
Minibatch accuracy: 20.0%
Validation accuracy: 61.5%
Minibatch loss at step 2500: 1.302774
Minibatch accuracy: 100.0%
Validation accuracy: 59.7%
Minibatch loss at step 3000: 3.024902
Minibatch accuracy: 80.0%
Validation accuracy: 60.7%
Test accuracy: 66.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
relu_units = 1024
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # WEIGHTS ONE
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, relu_units]))
  biases1 = tf.Variable(tf.zeros([relu_units]))

  # WEIGHTS TWO
  weights2 = tf.Variable( tf.truncated_normal([relu_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu_layer= tf.nn.relu(logits1)
  # Dropout on the hidden layer (RELU) !!!!!!!!!!!!!!!!!!!!!!!
  relu_dropout = tf.nn.dropout(relu_layer,keep_prob=0.5)

  logits2 = tf.matmul(relu_dropout, weights2) + biases2


  loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2))
 
  # Loss function with L2 Regularization with beta=0.01
  regularizers = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
  loss = tf.reduce_mean(loss + beta * regularizers)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training
  train_prediction = tf.nn.softmax(logits2)
  
  # Predictions for the validation data
  logits1 = tf.matmul(tf_valid_dataset, weights1) + biases1
  relu_layer= tf.nn.relu(logits1)
  logits2 = tf.matmul(relu_layer, weights2) + biases2

  valid_prediction = tf.nn.softmax(logits2)

  # Predictions for the test data
  logits1 = tf.matmul(tf_test_dataset, weights1) + biases1
  relu_layer= tf.nn.relu(logits1)
  logits2 = tf.matmul(relu_layer, weights2) + biases2

  test_prediction = tf.nn.softmax(logits2)

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = { tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict )
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy( predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3684.986328
Minibatch accuracy: 11.7%
Validation accuracy: 33.4%
Minibatch loss at step 500: 21.444471
Minibatch accuracy: 79.7%
Validation accuracy: 83.4%
Minibatch loss at step 1000: 0.896096
Minibatch accuracy: 85.2%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 0.790769
Minibatch accuracy: 85.9%
Validation accuracy: 83.6%
Minibatch loss at step 2000: 0.839780
Minibatch accuracy: 80.5%
Validation accuracy: 82.7%
Minibatch loss at step 2500: 0.804023
Minibatch accuracy: 81.2%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 0.817989
Minibatch accuracy: 82.0%
Validation accuracy: 83.3%
Test accuracy: 89.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


## changing learning rate

In [59]:
relu_units = 1024
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # WEIGHTS ONE
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, relu_units]))
  biases1 = tf.Variable(tf.zeros([relu_units]))

  # WEIGHTS TWO
  weights2 = tf.Variable( tf.truncated_normal([relu_units, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu_layer= tf.nn.relu(logits1)
  # Dropout on the hidden layer (RELU) !!!!!!!!!!!!!!!!!!!!!!!
  relu_dropout = tf.nn.dropout(relu_layer,keep_prob=0.5)

  logits2 = tf.matmul(relu_dropout, weights2) + biases2


  loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2))
 
  # Loss function with L2 Regularization with beta=0.01
  regularizers = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
  loss = tf.reduce_mean(loss + beta * regularizers)

  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.8, global_step, 1000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training
  train_prediction = tf.nn.softmax(logits2)
  
  # Predictions for the validation data
  logits1 = tf.matmul(tf_valid_dataset, weights1) + biases1
  relu_layer= tf.nn.relu(logits1)
  logits2 = tf.matmul(relu_layer, weights2) + biases2

  valid_prediction = tf.nn.softmax(logits2)

  # Predictions for the test data
  logits1 = tf.matmul(tf_test_dataset, weights1) + biases1
  relu_layer= tf.nn.relu(logits1)
  logits2 = tf.matmul(relu_layer, weights2) + biases2

  test_prediction = tf.nn.softmax(logits2)

In [60]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = { tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict )
     
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy( predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3638.842041
Minibatch accuracy: 10.9%
Validation accuracy: 36.7%
Minibatch loss at step 500: 2.005723
Minibatch accuracy: 75.0%
Validation accuracy: 82.7%
Minibatch loss at step 1000: 0.834691
Minibatch accuracy: 82.8%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 0.876457
Minibatch accuracy: 82.8%
Validation accuracy: 82.2%
Minibatch loss at step 2000: 0.881557
Minibatch accuracy: 81.2%
Validation accuracy: 81.3%
Minibatch loss at step 2500: 0.899701
Minibatch accuracy: 78.1%
Validation accuracy: 82.3%
Minibatch loss at step 3000: 0.891609
Minibatch accuracy: 78.9%
Validation accuracy: 82.8%
Test accuracy: 88.9%


# Multilayer approach 4 layers

In [12]:
import math 
relu_units1 = 1024
relu_units2 = 512
relu_units3 = 216

batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # WEIGHTS ONE
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, relu_units1], stddev=math.sqrt(2.0/(image_size*image_size))))
  biases1 = tf.Variable(tf.zeros([relu_units1]))

  # WEIGHTS TWO
  weights2 = tf.Variable( tf.truncated_normal([relu_units1, relu_units2], stddev=math.sqrt(2.0/relu_units1)))
  biases2 = tf.Variable(tf.zeros([relu_units2]))

  # WEIGHTS THREE
  weights3 = tf.Variable( tf.truncated_normal([relu_units2, relu_units3], stddev=math.sqrt(2.0/relu_units2)))
  biases3 = tf.Variable(tf.zeros([relu_units3]))

  # WEIGHTS FOUR
  weights4 = tf.Variable( tf.truncated_normal([relu_units3, num_labels], stddev=math.sqrt(2.0/relu_units3)))
  biases4 = tf.Variable(tf.zeros([num_labels]))
   
    
  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu_layer1= tf.nn.relu(logits1)
  #relu_dropout1 = tf.nn.dropout(relu_layer1,keep_prob=0.5)
    
  logits2 = tf.matmul(relu_layer1, weights2) + biases2
  relu_layer2= tf.nn.relu(logits2)
    
  logits3 = tf.matmul(relu_layer2, weights3) + biases3
  relu_layer3= tf.nn.relu(logits3)
    
  logits4 = tf.matmul(relu_layer3, weights4) + biases4


  loss = tf.reduce_mean( 
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits4))
 
  # Loss function with L2 Regularization with beta=0.01
  regularizers = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4)
  loss = tf.reduce_mean(loss + beta * regularizers)

  # Optimizer.
  #global_step = tf.Variable(0)  # count the number of steps taken.
  #learning_rate = tf.train.exponential_decay(0.8, global_step, 1000, 0.96, staircase=True)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training
  train_prediction = tf.nn.softmax(logits4)
  
  # Predictions for the validation data
  logits1 = tf.matmul(tf_valid_dataset, weights1) + biases1
  relu_layer1= tf.nn.relu(logits1)
  logits2 = tf.matmul(relu_layer1, weights2) + biases2
  relu_layer2= tf.nn.relu(logits2)
  logits3 = tf.matmul(relu_layer2, weights3) + biases3
  relu_layer3= tf.nn.relu(logits3)
  logits4 = tf.matmul(relu_layer3, weights4) + biases4

  valid_prediction = tf.nn.softmax(logits4)

  # Predictions for the test data
  logits1 = tf.matmul(tf_test_dataset, weights1) + biases1
  relu_layer1= tf.nn.relu(logits1)
  logits2 = tf.matmul(relu_layer1, weights2) + biases2
  relu_layer2= tf.nn.relu(logits2)
  logits3 = tf.matmul(relu_layer2, weights3) + biases3
  relu_layer3= tf.nn.relu(logits3)
  logits4 = tf.matmul(relu_layer3, weights4) + biases4

  test_prediction = tf.nn.softmax(logits4)

In [13]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = { tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict )
     
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy( predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.980942
Minibatch accuracy: 11.7%
Validation accuracy: 35.0%
Minibatch loss at step 500: 0.982470
Minibatch accuracy: 79.7%
Validation accuracy: 83.1%
Minibatch loss at step 1000: 0.792676
Minibatch accuracy: 84.4%
Validation accuracy: 82.6%
Minibatch loss at step 1500: 0.790650
Minibatch accuracy: 85.2%
Validation accuracy: 82.2%
Minibatch loss at step 2000: 0.874707
Minibatch accuracy: 80.5%
Validation accuracy: 82.5%
Minibatch loss at step 2500: 0.830478
Minibatch accuracy: 79.7%
Validation accuracy: 83.4%
Minibatch loss at step 3000: 0.845735
Minibatch accuracy: 80.5%
Validation accuracy: 83.7%
Test accuracy: 90.2%


In [14]:
import math as math

In [18]:
batch_size = 128
beta = 0.001

hidden_nodes_1 = 1024
hidden_nodes_2 = int(hidden_nodes_1 * 0.5)
hidden_nodes_3 = int(hidden_nodes_1 * np.power(0.5, 2))
hidden_nodes_4 = int(hidden_nodes_1 * np.power(0.5, 3))
hidden_nodes_5 = int(hidden_nodes_1 * np.power(0.5, 4))

graph = tf.Graph()
with graph.as_default():

    '''Input Data'''
    # For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    '''Variables'''
    # Hidden RELU layer 1
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1], stddev=math.sqrt(2.0/(image_size*image_size))))
    biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))

    # Hidden RELU layer 2
    weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], stddev=math.sqrt(2.0/hidden_nodes_1)))
    biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
    
    # Hidden RELU layer 3
    weights_3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, hidden_nodes_3], stddev=math.sqrt(2.0/hidden_nodes_2)))
    biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))
    
    # Hidden RELU layer 4
    weights_4 = tf.Variable(tf.truncated_normal([hidden_nodes_3, hidden_nodes_4], stddev=math.sqrt(2.0/hidden_nodes_3)))
    biases_4 = tf.Variable(tf.zeros([hidden_nodes_4]))
    
    # Hidden RELU layer 5
    weights_5 = tf.Variable(tf.truncated_normal([hidden_nodes_4, hidden_nodes_5], stddev=math.sqrt(2.0/hidden_nodes_4)))
    biases_5 = tf.Variable(tf.zeros([hidden_nodes_5]))
    
    # Output layer
    weights_6 = tf.Variable(tf.truncated_normal([hidden_nodes_5, num_labels], stddev=math.sqrt(2.0/hidden_nodes_5)))
    biases_6 = tf.Variable(tf.zeros([num_labels]))
    
    '''Training computation'''
    
    # Hidden RELU layer 1
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden_layer_1 = tf.nn.relu(logits_1)
    # Dropout on hidden layer: RELU layer
    keep_prob = tf.placeholder("float")
    hidden_layer_1_dropout = tf.nn.dropout(hidden_layer_1, keep_prob)
    
    
    # Hidden RELU layer 2
    logits_2 = tf.matmul(hidden_layer_1_dropout, weights_2) + biases_2
    hidden_layer_2 = tf.nn.relu(logits_2)
    # Dropout on hidden layer: RELU layer
    hidden_layer_2_dropout = tf.nn.dropout(hidden_layer_2, keep_prob)
    
    # Hidden RELU layer 3
    logits_3 = tf.matmul(hidden_layer_2_dropout, weights_3) + biases_3
    hidden_layer_3 = tf.nn.relu(logits_3)
    # Dropout on hidden layer: RELU layer
    hidden_layer_3_dropout = tf.nn.dropout(hidden_layer_3, keep_prob)
    
    # Hidden RELU layer 4
    logits_4 = tf.matmul(hidden_layer_3_dropout, weights_4) + biases_4
    hidden_layer_4 = tf.nn.relu(logits_4)
    # Dropout on hidden layer: RELU layer

    hidden_layer_4_dropout = tf.nn.dropout(hidden_layer_4, keep_prob)
    
    # Hidden RELU layer 5
    logits_5 = tf.matmul(hidden_layer_4_dropout, weights_5) + biases_5
    hidden_layer_5 = tf.nn.relu(logits_5)
    # Dropout on hidden layer: RELU layer
    hidden_layer_5_dropout = tf.nn.dropout(hidden_layer_5, keep_prob)
    
    # Output layer
    logits_6 = tf.matmul(hidden_layer_5_dropout, weights_6) + biases_6 
    
    # Normal loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_6))
    # Loss function with L2 Regularization with decaying learning rate beta=0.5
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + \
                   tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4) + \
                   tf.nn.l2_loss(weights_5) + tf.nn.l2_loss(weights_6)
    loss = tf.reduce_mean(loss + beta * regularizers)

    '''Optimizer'''
    # Decaying learning rate
    global_step = tf.Variable(0)  # count the number of steps taken.
    start_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_6)
    
    # Predictions for validation 
    valid_logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    valid_relu_1 = tf.nn.relu(valid_logits_1)
    
    valid_logits_2 = tf.matmul(valid_relu_1, weights_2) + biases_2
    valid_relu_2 = tf.nn.relu(valid_logits_2)
    
    valid_logits_3 = tf.matmul(valid_relu_2, weights_3) + biases_3
    valid_relu_3 = tf.nn.relu(valid_logits_3)
    
    valid_logits_4 = tf.matmul(valid_relu_3, weights_4) + biases_4
    valid_relu_4 = tf.nn.relu(valid_logits_4)
    
    valid_logits_5 = tf.matmul(valid_relu_4, weights_5) + biases_5
    valid_relu_5 = tf.nn.relu(valid_logits_5)
    
    valid_logits_6 = tf.matmul(valid_relu_5, weights_6) + biases_6
    
    valid_prediction = tf.nn.softmax(valid_logits_6)
    
    # Predictions for test
    test_logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    test_relu_1 = tf.nn.relu(test_logits_1)
    
    test_logits_2 = tf.matmul(test_relu_1, weights_2) + biases_2
    test_relu_2 = tf.nn.relu(test_logits_2)
    
    test_logits_3 = tf.matmul(test_relu_2, weights_3) + biases_3
    test_relu_3 = tf.nn.relu(test_logits_3)
    
    test_logits_4 = tf.matmul(test_relu_3, weights_4) + biases_4
    test_relu_4 = tf.nn.relu(test_logits_4)
    
    test_logits_5 = tf.matmul(test_relu_4, weights_5) + biases_5
    test_relu_5 = tf.nn.relu(test_logits_5)
    
    test_logits_6 = tf.matmul(test_relu_5, weights_6) + biases_6
    
    test_prediction = tf.nn.softmax(test_logits_6)
    

In [20]:
num_steps = 15000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))


Initialized
Minibatch loss at step 0: 5.39243412018
Minibatch accuracy: 8.6
Validation accuracy: 9.3
Minibatch loss at step 500: 1.89418411255
Minibatch accuracy: 77.3
Validation accuracy: 82.2
Minibatch loss at step 1000: 1.34887290001
Minibatch accuracy: 82.8
Validation accuracy: 84.4
Minibatch loss at step 1500: 1.161049366
Minibatch accuracy: 82.8
Validation accuracy: 84.4
Minibatch loss at step 2000: 1.0915760994
Minibatch accuracy: 78.9
Validation accuracy: 84.8
Minibatch loss at step 2500: 0.962060451508
Minibatch accuracy: 82.0
Validation accuracy: 85.7
Minibatch loss at step 3000: 0.938680291176
Minibatch accuracy: 82.0
Validation accuracy: 85.7
Minibatch loss at step 3500: 0.860002636909
Minibatch accuracy: 81.2
Validation accuracy: 85.9
Minibatch loss at step 4000: 0.919064760208
Minibatch accuracy: 80.5
Validation accuracy: 86.4
Minibatch loss at step 4500: 0.817328333855
Minibatch accuracy: 85.9
Validation accuracy: 86.1
Minibatch loss at step 5000: 0.871923565865
Minibatc